https://arxiv.org/pdf/1810.04805.pdf

In [1]:
import os
os.sys.path.append('..')

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [29]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import collections
import logging
import json
import math
import os
import random
import six
from tqdm import tqdm_notebook as tqdm

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

import tokenization
from modeling import BertConfig, BertForMaskedLanguageModelling
from optimization import BERTAdam


In [4]:

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s', 
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

# Args

In [5]:
parser = argparse.ArgumentParser()

## Required parameters
parser.add_argument("--data_dir",
                    default=None,
                    type=str,
                    required=True,
                    help="The input data dir. Should contain the .tsv files (or other data files) for the task.")
parser.add_argument("--bert_config_file",
                    default=None,
                    type=str,
                    required=True,
                    help="The config json file corresponding to the pre-trained BERT model. \n"
                         "This specifies the model architecture.")
parser.add_argument("--task_name",
                    default=None,
                    type=str,
                    required=True,
                    help="The name of the task to train.")
parser.add_argument("--vocab_file",
                    default=None,
                    type=str,
                    required=True,
                    help="The vocabulary file that the BERT model was trained on.")
parser.add_argument("--output_dir",
                    default=None,
                    type=str,
                    required=True,
                    help="The output directory where the model checkpoints will be written.")

## Other parameters
parser.add_argument("--init_checkpoint",
                    default=None,
                    type=str,
                    help="Initial checkpoint (usually from a pre-trained BERT model).")
parser.add_argument("--do_lower_case",
                    default=False,
                    action='store_true',
                    help="Whether to lower case the input text. True for uncased models, False for cased models.")
parser.add_argument("--max_seq_length",
                    default=128,
                    type=int,
                    help="The maximum total input sequence length after WordPiece tokenization. \n"
                         "Sequences longer than this will be truncated, and sequences shorter \n"
                         "than this will be padded.")
parser.add_argument("--do_train",
                    default=False,
                    action='store_true',
                    help="Whether to run training.")
parser.add_argument("--do_eval",
                    default=False,
                    action='store_true',
                    help="Whether to run eval on the dev set.")
parser.add_argument("--train_batch_size",
                    default=32,
                    type=int,
                    help="Total batch size for training.")
parser.add_argument("--eval_batch_size",
                    default=8,
                    type=int,
                    help="Total batch size for eval.")
parser.add_argument("--learning_rate",
                    default=5e-5,
                    type=float,
                    help="The initial learning rate for Adam.")
parser.add_argument("--num_train_epochs",
                    default=3.0,
                    type=float,
                    help="Total number of training epochs to perform.")
parser.add_argument("--warmup_proportion",
                    default=0.1,
                    type=float,
                    help="Proportion of training to perform linear learning rate warmup for. "
                         "E.g., 0.1 = 10%% of training.")
parser.add_argument("--no_cuda",
                    default=False,
                    action='store_true',
                    help="Whether not to use CUDA when available")
parser.add_argument("--local_rank",
                    type=int,
                    default=-1,
                    help="local_rank for distributed training on gpus")
parser.add_argument('--seed', 
                    type=int, 
                    default=42,
                    help="random seed for initialization")
parser.add_argument('--gradient_accumulation_steps',
                    type=int,
                    default=1,
                    help="Number of updates steps to accumualte before performing a backward/update pass.") 

_StoreAction(option_strings=['--gradient_accumulation_steps'], dest='gradient_accumulation_steps', nargs=None, const=None, default=1, type=<class 'int'>, choices=None, help='Number of updates steps to accumualte before performing a backward/update pass.', metavar=None)

- BERT_BASE_DIR/* is from pretrained model
- train-v1.1.json etc are training data e.g. https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json


In [6]:
experiment_name = 'erotic_uncased_4_tied_mlm'

argv = """
--task_name lm \
--data_dir {DATA_DIR} \
--vocab_file {BERT_BASE_DIR}/vocab.txt \
--bert_config_file {BERT_BASE_DIR}/bert_config.json \
--init_checkpoint {BERT_BASE_DIR}/pytorch_model.bin \
--do_train \
--do_eval \
--gradient_accumulation_steps 2 \
--train_batch_size 24 \
--learning_rate 3e-5 \
--num_train_epochs 3.0 \
--max_seq_length 128 \
--output_dir ../outputs/{name}/
""".format(
    BERT_BASE_DIR='../data/weights/cased_L-12_H-768_A-12',
    DATA_DIR='../data/input/erotic_gutenberg',
    name=experiment_name
).replace('\n', '').split(' ')
print(argv)
args = parser.parse_args(argv)

['--task_name', 'lm', '--data_dir', '../data/input/erotic_gutenberg', '--vocab_file', '../data/weights/cased_L-12_H-768_A-12/vocab.txt', '--bert_config_file', '../data/weights/cased_L-12_H-768_A-12/bert_config.json', '--init_checkpoint', '../data/weights/cased_L-12_H-768_A-12/pytorch_model.bin', '--do_train', '--do_eval', '--gradient_accumulation_steps', '2', '--train_batch_size', '24', '--learning_rate', '3e-5', '--num_train_epochs', '3.0', '--max_seq_length', '128', '--output_dir', '../outputs/erotic_uncased_4_tied_mlm/']


# Init

In [7]:
if args.local_rank == -1 or args.no_cuda:
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    n_gpu = torch.cuda.device_count()
else:
    device = torch.device("cuda", args.local_rank)
    n_gpu = 1
    # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.distributed.init_process_group(backend='nccl')
logger.info("device %s n_gpu %d distributed training %r", device, n_gpu, bool(args.local_rank != -1))

if args.gradient_accumulation_steps < 1:
    raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(
                        args.gradient_accumulation_steps))

args.train_batch_size = int(args.train_batch_size / args.gradient_accumulation_steps)

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if n_gpu > 0:
    torch.cuda.manual_seed_all(args.seed)

if not args.do_train and not args.do_eval:
    raise ValueError("At least one of `do_train` or `do_eval` must be True.")

11/11/2018 18:18:59 - INFO - __main__ -   device cuda n_gpu 1 distributed training False


In [8]:
bert_config = BertConfig.from_json_file(args.bert_config_file)

if args.max_seq_length > bert_config.max_position_embeddings:
    raise ValueError(
        "Cannot use sequence length {} because the BERT model was only trained up to sequence length {}".format(
        args.max_seq_length, bert_config.max_position_embeddings))

In [9]:
if os.path.exists(args.output_dir) and os.listdir(args.output_dir):
    print("Output directory ({}) already exists and is not empty.".format(args.output_dir))
os.makedirs(args.output_dir, exist_ok=True)

In [10]:
save_path = os.path.join(args.output_dir, 'state_dict.pkl')

# Load data v2

## Helpers

In [11]:
def notqdm(it, *a, **k):
    return it

In [12]:
# from https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/run_classifier.py

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id, label_weights):
        self.input_ids = input_ids # inputs tokens with 103 for mask
        self.input_mask = input_mask # 0 for padding, 1 otherwise
        self.segment_ids = segment_ids # which sentance it's in
        self.label_id = label_id # labels, true tokens
        self.label_weights = label_weights


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()
        

class LMProcessor(DataProcessor):
    """Processor for language modelling."""

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            open(os.path.join(data_dir, "train.txt")).read(), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            open(os.path.join(data_dir, "val.txt")).read(), "dev")

    def get_labels(self):
        """See base class."""
        return list(tokenizer.vocab.keys())

    def _create_examples(self, lines, set_type, window_size=args.max_seq_length, tqdm=tqdm):
        """Creates examples for the training and dev sets."""
        tokens = []
        for line in tqdm(lines.split('\n\n'), desc='tokenising'):
            line = tokenization.convert_to_unicode(line)
            token = tokenizer.tokenize(line)
            tokens += token
        
        context_list = []
        target_list = []
        examples = []
        for i, start_idx in tqdm(list(enumerate(range(len(tokens) - window_size - 1))), desc='chunking'):
            guid = "%s-%s" % (set_type, i)
            context = tokens[start_idx : start_idx + window_size]
            target = tokens[start_idx + window_size]
            examples.append(
                InputExample(guid=guid, text_a=context, text_b=None, label=target))
            
        return examples



In [13]:

def convert_tokens_to_features(examples, label_list, max_seq_length, tokenizer, tqdm=tqdm):
    """Loads a data file into a list of `InputBatch`s."""

    label_map = {}
    for (i, label) in enumerate(label_list):
        label_map[label] = i

    features = []
    for (ex_index, example) in tqdm(list(enumerate(examples))):
        tokens_a = example.text_a

        tokens_b = None
        if example.text_b:
            tokens_b = example.text_b

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[0:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = []
        segment_ids = []
        tokens.append("[CLS]")
        segment_ids.append(0)
        for token in tokens_a:
            tokens.append(token)
            segment_ids.append(0)
        tokens.append("[SEP]")
        segment_ids.append(0)

        if tokens_b:
            for token in tokens_b:
                tokens.append(token)
                segment_ids.append(1)
            tokens.append("[SEP]")
            segment_ids.append(1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)
        
        # see https://github.com/google-research/bert/blob/d8014ef72/create_pretraining_data.py#L363
        label_weights = np.random.rand(len(input_ids))<0.10 # 10% change of masking
        label_keep = (np.random.rand(len(input_ids))<0.10) * label_weights # 10% chance of keeping
        label_switch = (np.random.rand(len(input_ids))<0.10) * label_weights # 10% chance of random word
        label_mask = label_weights * (1-label_keep) * (1-label_keep)
        
        switched_ids = np.random.randint(low=0, high=len(tokenizer.vocab)-1, size=(len(input_ids),))
        
        input_ids_masked = np.array(input_ids.copy())
        input_ids_masked[label_switch==1] = switched_ids[label_switch==1]
        input_ids_masked[label_mask==1] = 103
        input_ids_masked = input_ids_masked.tolist()
        
        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        assert len(input_ids_masked) == max_seq_length
        
#         label_weights[torch.Tensor(input_ids)<103] = 0 # Don't mask CLS, PAD, ETC
#         input_ids_masked = input_ids.copy()
#         for i in range(len(label_weights)):
#             if label_weights[i]:
#                 if np.random.rand() < 0.8:
#                     input_ids_masked[i] = 103
#                 else:
#                     if np.random.rand() < 0.5:
#                         input_ids_masked[i] = np.random.randint(0, len(tokenizer.vocab)-1)
        
#         input_ids_masked[lms_masked==1] = 103
        # TODO 1% of the time replace masked ones with random work
        # TODO 1% of the time keep old one

#         if ex_index < 3:
#             logger.info("*** Example ***")
#             logger.info("guid: %s" % (example.guid))
#             logger.info("tokens: %s" % " ".join(
#                     [tokenization.printable_text(x) for x in tokens]))
#             logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
#             logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
#             logger.info(
#                     "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
#             logger.info("label: %s (id = %d)\n" % (example.label, label_id))

        features.append(
                InputFeatures(
                        input_ids=input_ids_masked,
                        input_mask=input_mask,
                        segment_ids=segment_ids,
                        label_id=input_ids,
                        label_weights=label_weights,))
    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

## Load

In [14]:
tokenizer = tokenization.FullTokenizer(
    vocab_file=args.vocab_file, do_lower_case=args.do_lower_case)

decoder = {v:k for k,v in tokenizer.wordpiece_tokenizer.vocab.items()}

In [15]:
# window_size                        = 128


In [16]:
processors = {
        "lm": LMProcessor,
}
    
task_name = args.task_name.lower()
if task_name not in processors:
        raise ValueError("Task not found: %s" % (task_name))

processor = processors[task_name]()
label_list = processor.get_labels()

In [17]:
train_examples = processor.get_train_examples(args.data_dir)
num_train_steps = int(
    len(train_examples) / args.train_batch_size * args.num_train_epochs)

In [18]:
train_features = convert_tokens_to_features(
    train_examples[::100], label_list, args.max_seq_length, tokenizer)

all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)
all_label_weights = torch.tensor([f.label_weights for f in train_features], dtype=torch.long)

In [19]:
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids, all_label_weights)
if args.local_rank == -1:
    train_sampler = RandomSampler(train_data)
else:
    train_sampler = DistributedSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size)

# Load model

In [30]:
model = BertForMaskedLanguageModelling(bert_config)
if args.init_checkpoint is not None:
    model.bert.load_state_dict(torch.load(args.init_checkpoint, map_location='cpu'))
    
# if os.path.isfile(save_path):
#     model.load_state_dict(torch.load(save_path, map_location='cpu'))
    
model.to(device)

if args.local_rank != -1:
    model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.local_rank],
                                                      output_device=args.local_rank)
elif n_gpu > 1:
    model = torch.nn.DataParallel(model)
    
model

BertForMaskedLanguageModelling(
  (bert): BertModel(
    (embeddings): BERTEmbeddings(
      (word_embeddings): Embedding(28996, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BERTLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BERTEncoder(
      (layer): ModuleList(
        (0): BERTLayer(
          (attention): BERTAttention(
            (self): BERTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BERTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BERTLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BERTInterm

# Opt

In [31]:
no_decay = ['bias', 'gamma', 'beta']
optimizer_parameters = [
    {'params': [p for n, p in model.named_parameters() if n not in no_decay], 'weight_decay_rate': 0.01},
    {'params': [p for n, p in model.named_parameters() if n in no_decay], 'weight_decay_rate': 0.0}
    ]

optimizer = BERTAdam(optimizer_parameters,
                     lr=args.learning_rate,
                     warmup=args.warmup_proportion,
                t_total=num_train_steps)

# Predict helpers

In [32]:
import re
from IPython.display import HTML, display

replace_list = [
    # punctuation
    ['\s\,', ','],
    ['\s\.', '.'],
    ['\s\:', ':'],
    ['\s\;', ';'],
    ['\s\!', '!'],
    ['\s\?', '?'],
    ["\s'\s?", "'"],
    ["\s-\s", "-"],
    ["\“\s", "“"],
    ["\s\”", "”"],
    ["\s\’", "’"],
    
    # tokenization
    ['\s?\<span([^\>]+)\>##', r'<span\1>'], # we might want to include html tags, to show e.g. the masks
    ['\s?##', ''],    
#     ['\[CLS\]\s?', ''],
#     ['\s?\[SEP\]\s?', ''],
#     ["\[PAD\]\s?", "_"],
#     [UNK]
#     [MASK]
#     [CLS]
]

def clean_decoded(tokens):
    s = ' '.join(tokens)
    for a, b in replace_list:
        s = re.sub(a, b, s)
    return s

def html_clean_decoded(input_ids, input_mask, logits, label_weights):
    """Format model outputs as html, with masked elements in red, with opacity indicating confidence."""
    log_probs = nn.LogSoftmax(-1)(logits).detach()
    prediction_idxs = log_probs.argmax(-1)
    # join masked an non masked
    y = input_ids *  (1 - label_weights) + prediction_idxs * label_weights
    yd = [decoder[hh.item()] for hh in y]
    html_yd = []
    for i in range(len(yd)):
        if not label_weights[i]:
            html_yd.append(yd[i])
        else:
            prob = log_probs[i][prediction_idxs[i]].exp()
            prob = prob/2 + 0.5
            html_yd.append('<span style="color: rgba(255,0,0,{})">{}</span>'.format(prob, yd[i]))
    return HTML(clean_decoded(html_yd))


In [33]:
# def predictions
def pad_seq(s1):
    x = '0 ' * (args.max_seq_length + 2 - len(tokenizer.tokenize(s1))) + s1
    return x


def predict_next_words(x, processor, tokenizer, n=10):
    x= pad_seq(x)
    ex = processor._create_examples(x, "train", tqdm=notqdm)[-1:]
    label_list = processor.get_labels()

    log_feats = convert_tokens_to_features(ex, label_list, args.max_seq_length, tokenizer, tqdm=notqdm)

    log_input_ids = torch.tensor([f.input_ids for f in log_feats], dtype=torch.long)
    log_input_mask = torch.tensor([f.input_mask for f in log_feats], dtype=torch.long)
    log_segment_ids = torch.tensor([f.segment_ids for f in log_feats], dtype=torch.long)
    log_label_ids = torch.tensor([f.label_id for f in log_feats], dtype=torch.long)
    log_label_weights = torch.tensor([f.label_weights for f in log_feats], dtype=torch.long)

    batch = [log_input_ids, log_input_mask, log_segment_ids, log_label_ids, log_label_weights]

    with torch.no_grad():
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids, label_weights = batch
        logits = model(input_ids, segment_ids, input_mask).detach()



    i = 0
    x = html_clean_decoded(input_ids=input_ids[i], input_mask=input_mask[i], logits=logits[i], label_weights=label_weights[i])
    
    return x



# Train 3

In [34]:
val_test="""In addition to this, the go-between should carefully note the behaviour of the woman, which if favourable would be as follows: She would address her with a smiling look, would seat herself close beside her, and ask her, "Where have you been? What have you been doing? Where did you dine? Where did you sleep? Where have you been sitting?" Moreover the woman would meet the go-between in lonely places and tell her stories there, would yawn contemplatively, draw long sighs, give her presents, remember her on occasions of festivals, dismiss her with a wish to see her again, and say to her jestingly, "Oh, well-speaking woman, why do you speak these bad words to me?" would discourse on the sin of her union with the man, would not tell her about any previous visits or conversations that she may have had with him, but wish to be asked about these, and lastly would laugh at the man's desire."""
display(predict_next_words(val_test, processor, tokenizer, n=20))

In [ ]:
model.train()
for _ in tqdm(range(int(args.num_train_epochs)), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    with tqdm(total=len(train_dataloader), desc='Iteration', mininterval=0.5) as prog:
        for step, batch in enumerate(train_dataloader):
            batch = tuple(t.to(device) for t in batch)
            input_ids, input_mask, segment_ids, label_ids, label_weights = batch
            
            loss, logits = model(input_ids, segment_ids, input_mask, label_ids, label_weights)
            if n_gpu > 1:
                loss = loss.mean() # mean() to average on multi-gpu.
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps
            loss.backward()
            tr_loss += loss.item()
            nb_tr_examples += input_ids.size(0)
            nb_tr_steps += 1
            if (step + 1) % args.gradient_accumulation_steps == 0:
                optimizer.step()    # We have accumulated enougth gradients
                model.zero_grad()
            prog.update(1)
            prog.desc = 'Iter. loss={:2.6f}'.format(tr_loss/nb_tr_examples)
            if step%1000==0:
                print(step, tr_loss/nb_tr_examples)
                display(predict_next_words(val_test, processor, tokenizer, n=10))
    
    torch.save(model.state_dict(), save_path)

global_step += 1

0 0.40383172035217285


1000 0.3824628845001117


2000 0.3342319631822781


3000 0.3078321982367521


4000 0.2894092392923434


...

6000 0.2633439270197097


7000 0.25355767033097065


8000 0.24516732770418348


9000 0.2378330940525342


10000 0.23133908374640988


In [ ]:
y = F.log_softmax(logits).argmax(-1).cpu().numpy()

In [ ]:
def clean_decoded(tokens):
    s = ' '.join(tokens)
    for a, b in replace_list:
        s = re.sub(a, b, s)
    return s

def html_clean_decoded(logits, input_mask, output, label_weights):
    """Format model outputs as html, with masked elements in red, with opacity indicating confidence."""
    log_probs = nn.LogSoftmax(-1)(logits).detach()
    prediction_idxs = log_probs.argmax(-1)
    # join masked an non masked
    y = input_ids *  (1 - label_weights) + prediction_idxs * label_weights
    yd = [decoder[hh.item()] for hh in y]
    html_yd = []
    for i in range(len(yd)):
        if not label_weights[i]:
            html_yd.append(yd[i])
        else:
            prob = log_probs[i][prediction_idxs[i]].exp()
            prob = prob/2 + 0.5
            html_yd.append('<span style="color: rgba(255,0,0,{})">{}</span>'.format(prob, yd[i]))
    return HTML(clean_decoded(html_yd))

In [ ]:
x = html_clean_decoded(input_ids[i], input_mask[i], logits[i], label_weights[i])

In [ ]:

log_probs = nn.LogSoftmax(-1)(logits).detach()
prediction_idxs = log_probs.argmax(-1)
# join masked an non masked
y = input_ids[0] *  (1 - label_weights[0]) + prediction_idxs[0] * label_weights[0]
yd = [decoder[hh.item()] for hh in y]
html_yd = []
for i in range(len(yd)):
    if not label_weights[0][i]:
        html_yd.append(yd[i])
    else:
        prob = logits[0][i][prediction_idxs[0][i]].exp()
        prob = prob/2 + 0.5
        html_yd.append('<span style="color: rgba(255,0,0,{})">{}</span>'.format(prob, yd[i]))
HTML(clean_decoded(html_yd))

In [ ]:
torch.save(model.state_dict(), save_path)

- TODO pred generator
- TODO sample outputs
- TODO check it works
- TODO isn't it a bit slow?
- TODO should I mirror the embedding weights?

## Language modelling loss...

In [ ]:
def to_one_hot(y_tensor, n_dims=None):
    """ Take integer y (tensor or variable) with n dims and convert it to 1-hot representation with n+1 dims. """
    y_tensor = y_tensor.type(torch.LongTensor).view(-1, 1)
    n_dims = n_dims if n_dims is not None else int(torch.max(y_tensor)) + 1
    y_one_hot = torch.zeros(y_tensor.size()[0], n_dims).scatter_(1, y_tensor, 1)
    y_one_hot = y_one_hot.view(*y.shape, -1)
    return y_one_hot


In [ ]:
# see https://github.com/google-research/bert/blob/d8014ef72/run_pretraining.py#L273
label_weights = (1-input_mask)
input_ids_1_hot = to_one_hot(input_ids, log_probs.shape[-1]).cuda()
per_example_loss = -(log_probs * input_ids_1_hot).sum(-1)
numerator = (label_weights.float() * per_example_loss).sum()
denominator = (label_weights.float()).sum() + 1e-5
loss = numerator / denominator
loss

In [ ]:
# Calc language masking loss
label_weights = (1-input_mask)
logits = outs * label_weights[:, :, None].float()
targets = input_ids * label_weights

logits = logits.view((-1, logits.shape[-1]))
targets = targets.view((-1,))

l = F.cross_entropy(logits, targets, reduce=False) * label_weights.float()
l.sum()/(input_mask==0).sum().float()


In [ ]:
# Calc language masking loss
label_weights = (1-input_mask)
logits = outs * label_weights[:, :, None].float()
targets = input_ids * label_weights

logits = logits.view((-1, logits.shape[-1]))
targets = targets.view((-1,))

logits = logits[targets!=0]
targets = targets[targets!=0]

F.cross_entropy(logits, targets)


In [ ]:
outs.shape

In [ ]:
input_mask.shape